In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_decision_forests as tfdf
from urllib.parse import unquote_to_bytes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# example data
data = [
  { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2D', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2E', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2F', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
   { 'str':'foo=1%27%20OR%201=1%2D%2G', 'attack':1 },
{ 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2H', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2I', 'attack':1 },
     { 'str':'foo=2', 'attack':0 },
     { 'str':'foo=3', 'attack':0 },
     { 'str':'foo=4', 'attack':0 },
     { 'str':'foo=5', 'attack':0 },
     { 'str':'foo=6', 'attack':0 },
     { 'str':'foo=7', 'attack':0 },
     { 'str':'foo=8', 'attack':0 },
     { 'str':'foo=9', 'attack':0 },
     { 'str':'foo=10', 'attack':0 },


]


#
# Prepare Data
#

maxlen = 1024

X = []
Y = []

def decode_query(str):
    return unquote_to_bytes(str.replace('+',' '))

for item in data:
    arr = decode_query(item['str'])[:maxlen]
    arrlen = len(arr)
    seq = [0] * maxlen
    for i in range(arrlen):
        seq[maxlen - arrlen + i] = arr[i]
    X.append(seq)
    Y.append(item['attack'])

#
# Build Model (Simple LSTM)
#
# Convert X and Y to NumPy arrays
X = np.array(X)
Y = np.array(Y)
#model = tf.keras.Sequential([
#    layers.Input(shape=(maxlen,), batch_size=1),
#    layers.Embedding(256, 32),
#    layers.randomforest(16),
#    layers.Dense(1, activation='sigmoid')])

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)


model = tfdf.keras.RandomForestModel()

# Add evaluation metrics
model.compile(
    metrics=["accuracy"]
)

model.fit(x=X_train, y=y_train)
# Predict on the test set
y_pred = model.predict(X_test)
print(y_pred)
print(y_train)
# Convert predictions to binary values (0 or 1)
y_pred = (y_pred > 0.11).astype(int) # Assuming binary classification

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

export_archive = tf.keras.export.ExportArchive()
export_archive.track(model)
export_archive.add_endpoint(
    name='serve',
    fn=model.call,
    input_signature=[tf.TensorSpec(shape=(1, maxlen), dtype=tf.float32)],
)
export_archive.write_out('model')

converter = tf.lite.TFLiteConverter.from_saved_model('model')

rf_model = converter.convert()

with open('rf.model', 'wb') as f:
    f.write(rf_model)

Use /tmp/tmpkcyekwjd as temporary training directory
Reading training dataset...


Training dataset read in 0:00:16.141555. Found 10 examples.
Training model...
Model trained in 0:00:00.378229
Compiling model...
Model compiled.


1/1 [==============================] - 4s 4s/step
[[0.        ]
 [0.13999994]
 [0.07999999]
 [0.07999999]
 [0.        ]
 [0.07999999]
 [0.22999986]
 [0.07999999]
 [0.13999994]
 [0.10666663]
 [0.07999999]]
[0 1 0 0 0 1 0 0 0 0]
Accuracy: 0.55
Precision: 0.33
Recall: 0.25
F1 Score: 0.29


AssertionError: Tried to export a function which references an 'untracked' resource. TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly. See the information below:
	Function name = b'__inference_signature_wrapper_call_125619'
	Captured Tensor = <ResourceHandle(name="simple_ml_model_ca272757-1cdb-4f83-8ff7-bb0bd61ff0ef", device="/job:localhost/replica:0/task:0/device:CPU:0", container="localhost", type="tensorflow_decision_forests::ops::YggdrasilModelResource", dtype and shapes : "[  ]")>
	Trackable referencing this tensor = <tensorflow_decision_forests.tensorflow.ops.inference.api._CompiledSimpleMLModelResource object at 0x7867b1f76c90>
	Internal Tensor = Tensor("125615:0", shape=(), dtype=resource)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_decision_forests as tfdf
from urllib.parse import unquote_to_bytes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
# example data
data = [
  { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2D', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2E', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2F', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
   { 'str':'foo=1%27%20OR%201=1%2D%2G', 'attack':1 },
{ 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2H', 'attack':1 },
      { 'str':'foo=1', 'attack':0 },
    { 'str':'foo=1%27%20OR%201=1%2D%2I', 'attack':1 },
     { 'str':'foo=2', 'attack':0 },
     { 'str':'foo=3', 'attack':0 },
     { 'str':'foo=4', 'attack':0 },
     { 'str':'foo=5', 'attack':0 },
     { 'str':'foo=6', 'attack':0 },
     { 'str':'foo=7', 'attack':0 },
     { 'str':'foo=8', 'attack':0 },
     { 'str':'foo=9', 'attack':0 },
     { 'str':'foo=10', 'attack':0 },


]


#
# Prepare Data
#

maxlen = 1024

X = []
Y = []

def decode_query(str):
    return unquote_to_bytes(str.replace('+',' '))

for item in data:
    arr = decode_query(item['str'])[:maxlen]
    arrlen = len(arr)
    seq = [0] * maxlen
    for i in range(arrlen):
        seq[maxlen - arrlen + i] = arr[i]
    X.append(seq)
    Y.append(item['attack'])


#
# Build Model (Simple LSTM)
#

model = tf.keras.Sequential([
    layers.Input(shape=(maxlen,), batch_size=1),
    layers.Embedding(256, 32),
    layers.LSTM(16),
    layers.Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#
# Train Model
#
start = time.time()
model.fit(np.asarray(X).astype(np.float32),
          np.asarray(Y).astype(np.float32),
          epochs=100, batch_size=1)
end = time.time()
print("training time", end - start) # time in seconds

predictions = model.predict(np.asarray(X).astype(np.float32))
predicted_classes = (predictions > 0.5).astype(int)
acc = accuracy_score(Y, predicted_classes)
print("Model accuracy:", acc)
#
# Save Model
#
model.fit(x=X_train, y=y_train)
# Predict on the test set
y_pred = model.predict(X_test)
# Convert predictions to binary values (0 or 1)
y_pred = (y_pred > 0.5).astype(int) # Assuming binary classification

accuracy = accuracy_score(Y, predicted_classes)
precision = precision_score(Y, predicted_classes)
recall = recall_score(Y, predicted_classes)
f1 = f1_score(Y, predicted_classes)


print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

export_archive = tf.keras.export.ExportArchive()
export_archive.track(model)
export_archive.add_endpoint(
    name='serve',
    fn=model.call,
    input_signature=[tf.TensorSpec(shape=(1, maxlen), dtype=tf.float32)],
)
export_archive.write_out('model')

converter = tf.lite.TFLiteConverter.from_saved_model('model')

rf_model = converter.convert()

with open('rf.model', 'wb') as f:
    f.write(rf_model)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (1, 1024, 32)               │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (1, 16)                     │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (1, 1)                      │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,345 (44.32 KB)

 Trainable params: 11,345 (44.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 231ms/step - accuracy: 0.6575 - loss: 0.6695
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 285ms/step - accuracy: 0.7645 - loss: 0.5062
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 232ms/step - accuracy: 0.6167 - loss: 0.3991
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 302ms/step - accuracy: 1.0000 - loss: 0.2204
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - accuracy: 1.0000 - loss: 0.2190
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 421ms/step - accuracy: 1.0000 - loss: 0.1421
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 229ms/step - accuracy: 1.0000 - loss: 0.0829
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 299ms/step - accuracy: 1.0000 - loss: 0.0728
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 270ms/step - accuracy: 1.0000 - loss: 0.0717
Epoch 10/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 251ms/step - accuracy: 1.0000 - loss: 0.0882
Epoch 11/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 228ms/step - accuracy: 1.0000 - loss: 0.0491
Epoch 12/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip list | grep tensorflow



tensorflow                         2.18.0
tensorflow-datasets                4.9.7
tensorflow_decision_forests        1.11.0
tensorflow-hub                     0.16.1
tensorflow-io-gcs-filesystem       0.37.1
tensorflow-metadata                1.13.1
tensorflow-probability             0.24.0
tensorflowjs                       4.22.0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
pip install tensorflow[and-cuda]

In [ ]:
pip show version tf_keras

Name: tf_keras
Version: 2.18.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: tensorflow
Required-by: tensorflow-hub, tensorflow_decision_forests, tensorflowjs


In [ ]:
pip show versions tensorflowjs


Name: tensorflowjs
Version: 4.22.0
Summary: 
Home-page: https://js.tensorflow.org/
Author: Google LLC
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: flax, importlib_resources, jax, jaxlib, packaging, six, tensorflow, tensorflow-decision-forests, tensorflow-hub, tf-keras
Required-by: 
